In [19]:
from dotenv import load_dotenv
from openai import AsyncOpenAI
from agents import Agent, Runner, trace, function_tool, OpenAIChatCompletionsModel, GuardrailFunctionOutput, OutputGuardrailTripwireTriggered, RunContextWrapper, output_guardrail
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content

In [2]:
load_dotenv(override=True)

True

In [6]:
openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set (and this is optional)")

if deepseek_api_key:
    print(f"DeepSeek API Key exists and begins {deepseek_api_key[:3]}")
else:
    print("DeepSeek API Key not set (and this is optional)")

if groq_api_key:
    print(f"Groq API Key exists and begins {groq_api_key[:4]}")
else:
    print("Groq API Key not set (and this is optional)")

OpenAI API Key exists and begins sk-proj-
Google API Key exists and begins AIzaSyA5
DeepSeek API Key exists and begins sk-
Groq API Key exists and begins gsk_


In [7]:
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
DEEPSEEK_BASE_URL = "https://api.deepseek.com/v1"
GROQ_BASE_URL = "https://api.groq.com/openai/v1"

In [8]:
deepseek_client = AsyncOpenAI(base_url=DEEPSEEK_BASE_URL, api_key=deepseek_api_key)
gemini_client = AsyncOpenAI(base_url=GEMINI_BASE_URL, api_key=google_api_key)

In [ ]:
sales_agent1 = Agent(
        name="OpenAI Sales Agent",
        instructions="You are a sales agent working for ComplAI, a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
                You write great sales emails that are likely to get a response.",
        model="gpt-4o-mini",
        handoff_description="A cold sales email has been written"
)

sales_agent2 =  Agent(
        name="DeepSeekSales Agent",
        instructions="You are a sales agent working for ComplAI, a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
                You write great sales emails that are likely to get a response.",
        model=OpenAIChatCompletionsModel(model="deepseek-chat", openai_client=deepseek_client),
        handoff_description="A cold sales email has been written"
)

sales_agent3  = Agent(
        name="Gemini Sales Agent",
        instructions="You are a sales agent working for ComplAI, a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
                You write great sales emails that are likely to get a response.",
        model=OpenAIChatCompletionsModel(model="gemini-2.0-flash", openai_client=gemini_client),
        handoff_description="A cold sales email has been written"
)

In [12]:
@function_tool
def send_email(subject: str, html_body: str) -> Dict[str, str]:
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("ed@edwarddonner.com")  # Change to your verified sender
    to_email = To("ed.donner@gmail.com")  # Change to your recipient
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content)
    mail_json = mail.get()
    response = sg.client.mail.send.post(request_body=mail_json)
    return {"status": "success"}

In [13]:
subject_writer = Agent(
        name="Email subject writer",
        instructions="You can write a subject for a cold sales email. You are given a message and you need to write a subject for an email that is likely to get a response.",
        model="gpt-4o-mini",
        handoff_description="A cold sales email subject has been written")

html_email_body_converter = Agent(
    name="HTML email body converter",   
    instructions="You can convert a text email body to an HTML email body. You are given a text email body which might have some markdown \
          and you need to convert it to an HTML email body with simple,clear, compelling layout and design.",
    model="gpt-4o-mini",
    handoff_description="An HTML email body has been written")

In [14]:
sales_manager = Agent(
    name="Sales Manager",
    instructions="You are a sales manager working for ComplAI. You use the tools given to you to generate cold sales emails. \
You never generate sales emails yourself; you always use the tools. \
You try all the tools at least once before choosing the best one. \
You pick the best email, and then you use the tool to write a subject for the chosen email, and another tool to convert the body of the chosen email to HTML. \
    Then you use the tool to send the email with that subject and HTML body to the user.",
    tools=[
        sales_agent1.as_tool(
            tool_name="sales_agent1",
            tool_description="Write a cold sales email",
        ),
        sales_agent2.as_tool(
            tool_name="sales_agent2",
            tool_description="Write a cold sales email",
        ),
        sales_agent3.as_tool(
            tool_name="sales_agent3",
            tool_description="Write a cold sales email",
        ),
        subject_writer.as_tool(
            tool_name="subject_writer",
            tool_description="Write a subject for a cold sales email",
        ),
        html_email_body_converter.as_tool(
            tool_name="html_email_body_converter",
            tool_description="Convert a text email body to an HTML email body",
        ),
        send_email
    ],
)

async def main():
    message = "Send a cold sales email addressed to Mike from Alice"

    with trace("Sales manager"):
        result = await Runner.run(sales_manager, message)

In [15]:
await main()

In [22]:
@output_guardrail
async def placeholder_guardrail(
    context: RunContextWrapper, agent: Agent, output: str
) -> GuardrailFunctionOutput:
    found_placeholders = any(word in output.lower() for word in ["[", "]", "your name"])

    return GuardrailFunctionOutput(
        output_info={
            "found_placeholders": found_placeholders,
        },
        tripwire_triggered=found_placeholders,
    )

In [26]:
subject_writer = Agent(
        name="Email subject writer",
        instructions="You can write a subject for a cold sales email. You are given a message and you need to write a subject for an email that is likely to get a response.",
        model="gpt-4o-mini",
        handoff_description="A cold sales email subject has been written",
        output_guardrails=[placeholder_guardrail])

html_email_body_converter = Agent(
    name="HTML email body converter",   
    instructions="You can convert a text email body to an HTML email body. You are given a text email body which might have some markdown \
          and you need to convert it to an HTML email body with simple,clear, compelling layout and design.",
    model="gpt-4o-mini",
    handoff_description="An HTML email body has been written",
    output_guardrails=[placeholder_guardrail])

In [27]:
sales_manager = Agent(
    name="Sales Manager",
    instructions="You are a sales manager working for ComplAI. You use the tools given to you to generate cold sales emails. \
You never generate sales emails yourself; you always use the tools. \
You try all the tools at least once before choosing the best one. \
You pick the best email, and then you use the tool to write a subject for the chosen email, and another tool to convert the body of the chosen email to HTML. \
    Then you use the tool to send the email with that subject and HTML body to the user.",
    tools=[
        sales_agent1.as_tool(
            tool_name="sales_agent1",
            tool_description="Write a cold sales email",
        ),
        sales_agent2.as_tool(
            tool_name="sales_agent2",
            tool_description="Write a cold sales email",
        ),
        sales_agent3.as_tool(
            tool_name="sales_agent3",
            tool_description="Write a cold sales email",
        ),
        subject_writer.as_tool(
            tool_name="subject_writer",
            tool_description="Write a subject for a cold sales email",
        ),
        html_email_body_converter.as_tool(
            tool_name="html_email_body_converter",
            tool_description="Convert a text email body to an HTML email body",
        ),
        send_email
    ],
)

async def main():
    message = "Send a cold sales email addressed to Mike from Alice"

    with trace("Sales manager"):
        result = await Runner.run(sales_manager, message)

In [28]:
await main()